In [3]:
# Run 'source ~/.profile'
import satpy
from satpy import Scene, find_files_and_readers
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pyresample import geometry
from pyproj import CRS
from satpy.composites import GenericCompositor
from satpy.writers import to_image
import os

sys.path.insert(0,'/home/cameron/Projects/')

#print(satpy.config.to_dict())
#print(satpy.available_readers())

#files_1 = find_files_and_readers(base_dir="/home/cameron/Dokumenter/Data/erie/erie_2023-03-01_1559Z-l1a.nc", reader='hypso1_l1a_nc')
#files_1 = find_files_and_readers(base_dir="/home/cameron/Dokumenter/Data/erie/", reader='hypso1_l1a_nc')
#files_1 = find_files_and_readers(base_dir="/home/cameron/Nedlastinger/erie_2023-03-01_1559Z-l1a.nc", reader='hypso1_l1a_nc')


In [6]:
satpy.config.set(config_path=['/home/cameron/Projects/'])

In [8]:
satpy.available_readers()

['abi_l1b',
 'abi_l1b_scmi',
 'abi_l2_nc',
 'acspo',
 'agri_fy4a_l1',
 'agri_fy4b_l1',
 'ahi_hrit',
 'ahi_hsd',
 'ahi_l1b_gridded_bin',
 'ahi_l2_nc',
 'amsr2_l1b',
 'amsr2_l2',
 'amsr2_l2_gaasp',
 'amsub_l1c_aapp',
 'atms_l1b_nc',
 'atms_sdr_hdf5',
 'avhrr_l1b_aapp',
 'avhrr_l1b_eps',
 'avhrr_l1c_eum_gac_fdr_nc',
 'cmsaf-claas2_l2_nc',
 'electrol_hrit',
 'epic_l1b_h5',
 'fci_l1c_nc',
 'fci_l2_nc',
 'generic_image',
 'geocat',
 'gerb_l2_hr_h5',
 'ghi_l1',
 'ghrsst_l2',
 'glm_l2',
 'goes-imager_hrit',
 'goes-imager_nc',
 'gpm_imerg',
 'hsaf_h5',
 'hy2_scat_l2b_h5',
 'iasi_l2',
 'iasi_l2_cdr_nc',
 'jami_hrit',
 'li_l2_nc',
 'maia',
 'meris_nc_sen3',
 'mhs_l1c_aapp',
 'mimicTPW2_comp',
 'mirs',
 'msi_safe',
 'msu_gsa_l1b',
 'mtsat2-imager_hrit',
 'mviri_l1b_fiduceo_nc',
 'mws_l1b_nc',
 'nucaps',
 'nwcsaf-geo',
 'nwcsaf-msg2013-hdf5',
 'nwcsaf-pps_nc',
 'oceancolorcci_l3_nc',
 'olci_l1b',
 'olci_l2',
 'omps_edr',
 'osisaf_nc',
 'safe_sar_l2_ocn',
 'sar-c_safe',
 'satpy_cf_nc',
 'seviri_l1b_

In [2]:
nc_file = '/home/cameron/Dokumenter/Data/erie/erie_2023-03-01_1559Z-l1a.nc'
points_file = '/home/cameron/Dokumenter/Data/erie/erie_2023-03-01_1559Z-bin3.points'
#scene_1 = Scene(filenames=[nc_file, points_file], reader='hypso1_l1a_nc')

print(points_file)

scene_1 = Scene(filenames=[points_file, nc_file], reader='hypso1_l1a_nc')

/home/cameron/Dokumenter/Data/erie/erie_2023-03-01_1559Z-bin3.points


Don't know how to open the following files: {'/home/cameron/Dokumenter/Data/erie/erie_2023-03-01_1559Z-bin3.points', '/home/cameron/Dokumenter/Data/erie/erie_2023-03-01_1559Z-l1a.nc'}


ValueError: No supported files found

In [6]:
datasets_1 = scene_1.available_dataset_names()

NameError: name 'scene_1' is not defined

In [5]:
# Don't need to load all the datasets just for testing
#scene_1.load(datasets_1)
scene_1.load(['latitude', 'longitude', 'band_80'])

Required file type '['hypso1_gcp_points_lat_lon']' not found or loaded for 'latitude'
Required file type '['hypso1_gcp_points_lat_lon']' not found or loaded for 'longitude'
Failed to load coordinates for 'DataID(name='band_80', wavelength=WavelengthRange(min=669, central=669, max=669, unit='nm'), modifiers=())'
Required file type '['hypso1_gcp_points_lat_lon']' not found or loaded for 'latitude'
Required file type '['hypso1_gcp_points_lat_lon']' not found or loaded for 'longitude'
The following datasets were not created and may require resampling to be generated: DataID(name='latitude', resolution=-999, modifiers=()), DataID(name='longitude', resolution=-999, modifiers=())


In [ ]:



grid_lats = scene_1['80'].attrs['area'].lats.data
grid_lons = scene_1['80'].attrs['area'].lons.data

# Is there a function that can do this? Possibly in pyresample.
lon_min = grid_lons.min()
lon_max = grid_lons.max()
lat_min = grid_lats.min()
lat_max = grid_lats.max()

bbox = (lon_min,lat_min,lon_max,lat_max)
#bbox = (-83.534546,41.356196,-82.359009,42.706660) # W. Lake Erie

print(bbox)

area_id = 'western_lake_erie'
proj_id = 'roi'
description = 'roi'
projection = CRS.from_epsg(4326)
width = 500
height = 1000
area_extent = list(bbox)

# Define area definition
area_def = geometry.AreaDefinition(area_id, proj_id, description, projection,  width, height, area_extent)


# Reading Area Definitions: https://satpy.readthedocs.io/en/stable/resample.html#store-area-definitions
#from pyresample import load_area
#area_def_loaded = load_area(filename)

# Resample to area_def
local_scene_1 = scene_1.resample(area_def, resampler='bilinear', fill_value=np.NaN)


print('Writing composites...')

gamma = 2

# Original capture composites

s = scene_1
compositor = GenericCompositor("overview")
composite = compositor([s['80'][:,::3], s['40'][:,::3], s['15'][:,::3]]) # Red, Green, Blue
#composite = composite[:,:,::-1] # correct for composite mirroring
img = to_image(composite[:,:,::-1]) 
img.invert([False, False, False])
img.stretch("linear")
img.gamma([gamma, gamma, gamma])
img.save('./composites/nc_scene1.png')


# Resampled capture composites

s = local_scene_1
compositor = GenericCompositor("overview")
composite = compositor([s['80'], s['40'], s['15']]) # Red, Green, Blue
img = to_image(composite)
img.invert([False, False, False])
img.stretch("linear")
img.gamma([gamma, gamma, gamma])
img.save('./composites/nc_resampled_scene1.png')


print('Done.')


In [17]:
scene_1['15'].to_numpy().min()

35.0343633604119

In [19]:
scene_1['longitude']

<xarray.DataArray (y: 956, x: 684)>
array([[-9193687.67468701, -9193595.32836108, -9193502.97760249, ...,
        -9129773.48944296, -9129678.12447796, -9129582.7550803 ],
       [-9193795.07226164, -9193702.72957986, -9193610.38246542, ...,
        -9129883.36868578, -9129788.00736493, -9129692.64161143],
       [-9193902.46490318, -9193810.12586555, -9193717.78239527, ...,
        -9129993.24299552, -9129897.88531882, -9129802.52320947],
       ...,
       [-9293799.77738666, -9293710.90393854, -9293622.02605776, ...,
        -9232250.62193279, -9232158.7298456 , -9232066.83332576],
       [-9293902.47373037, -9293813.6039264 , -9293724.72968977, ...,
        -9232355.79994469, -9232263.91150166, -9232172.01862596],
       [-9294005.16514098, -9293916.29898117, -9293827.4283887 , ...,
        -9232460.97302351, -9232369.08822462, -9232277.19899308]])
Dimensions without coordinates: y, x
Attributes:
    standard_name:        longitude
    start_time:           2023-03-01 15:59:00
    end_time:             2023-03-01 15:59:00
    reader:               hypso1_l1a_nc
    name:                 longitude
    modifiers:            ()
    _satpy_id:            DataID(name='longitude', modifiers=())
    long_name:            longitude
    units:                degrees_east
    ancillary_variables:  []